<h3 align="center">Status : <span class="badge"><b>En cours</b></span></h3>

<h1 align="center">RL GRPO</h1>

---

<h1 align="center">Training a small Unimarc reasoner with RL</h1>

# Requirements

In [1]:
import torch
print("PyTorch:")
print("PyTorch version is:" + torch.__version__)
print("PyTorch is working with CUDA" if torch.cuda.is_available() else "Error! It is not working correctly")
print("The GPU model is: "+ torch.cuda.get_device_name(0))

PyTorch:
PyTorch version is:2.7.0+cu126
PyTorch is working with CUDA
The GPU model is: Tesla T4


In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -U --quiet datasets transformers trl huggingface_hub accelerate peft --use-deprecated=legacy-resolver
#tensordock: pip install flash-attn tf-keras

In [12]:
from huggingface_hub import login, whoami

login(
  token="..." # with write permissions
)
whoami()

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'type': 'user',
 'id': '63256c1fa3f07c8e168c0d47',
 'name': 'Geraldine',
 'fullname': 'Géraldine Geoffroy',
 'email': 'grldn.geoffroy@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'periodEnd': 1748735999,
 'isPro': False,
 'avatarUrl': '/avatars/9cb069e5e90930e818ebe69300cd35d8.svg',
 'orgs': [{'type': 'org',
   'id': '665f255b175693a15893b7a1',
   'name': 'discord-community',
   'fullname': 'Hugging Face Discord Community',
   'email': 'lunarflu@gmail.com',
   'canPay': False,
   'periodEnd': 1748735999,
   'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/6340651b388c3fa40f9a5bc0/j6Vb_hutYuKRcQgMaDTAt.png',
   'roleInOrg': 'read',
   'isEnterprise': False}],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'write_hf_token',
   'role': 'write',
   'createdAt': '2024-11-08T10:49:12.891Z'}}}

# Load model

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [14]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    #attn_implementation="flash_attention_2"
)
model.config.sliding_window = None

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [15]:
model.config.sliding_window = None
device = "cuda" # for GPU usage or "cpu" for CPU usage
model = model.to(device)
model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [16]:
# !Important
tokenizer.padding_side = "left"

## Inspect tokenizer

In [7]:
len(tokenizer)

151665

In [16]:
print(f"**EOS**\nEOS token: {tokenizer.eos_token}\n- EOS token id: {tokenizer.eos_token_id}\n\n**PAD**\nPAD token: {tokenizer.pad_token}\n- PAD token id: {tokenizer.pad_token_id}")

**EOS**
EOS token: <|im_end|>
- EOS token id: 151645

**PAD**
PAD token: <|endoftext|>
- PAD token id: 151643


In [18]:
tokenizer("the sky is blue", return_tensors="pt").to(device)

{'input_ids': tensor([[ 1782, 12884,   374,  6303]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}

In [19]:
input_ids = tokenizer("the sky is blue", return_tensors="pt").to(device).input_ids[0]

In [20]:
tokenizer.decode(input_ids)

'the sky is blue'

In [21]:
if hasattr(tokenizer, "chat_template"):
    print("Current chat template:", tokenizer.chat_template)

Current chat template: {%- if tools %}
    {{- '<|im_start|>system\n' }}
    {%- if messages[0]['role'] == 'system' %}
        {{- messages[0]['content'] }}
    {%- else %}
        {{- 'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.' }}
    {%- endif %}
    {{- "\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>" }}
    {%- for tool in tools %}
        {{- "\n" }}
        {{- tool | tojson }}
    {%- endfor %}
    {{- "\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call><|im_end|>\n" }}
{%- else %}
    {%- if messages[0]['role'] == 'system' %}
        {{- '<|im_start|>system\n' + messages[0]['content'] + '<|im_end|>\n' }}
    {%- else %}
        {{- '<|im_start|>system\nYou are Qwe

## Inference

### Without applying chat templating

In [37]:
prompt = "<|im_start|>system: You are a helpful assistant<|im_end|><|im_start|>user: complete this sentence 'the sky is blue and '<|im_end|><|im_start|>assistant: "
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [38]:
outputs = model.generate(**inputs,
                         max_new_tokens = 20,
                         use_cache = True,)

In [39]:
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<|im_start|>system: You are a helpful assistant<|im_end|><|im_start|>user: complete this sentence 'the sky is blue and '<|im_end|><|im_start|>assistant:  the sky is blue and the clouds are white.<|im_end|>


### With applying chat templating

In [38]:
SYSTEM_PROMPT_0 = """
You are an expert in Unimarc/XML bibliographic records.
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

SYSTEM_PROMPT_0_SMALL = """
You are an expert in Unimarc/XML bibliographic records.
Respond in the following format:
### Resoaning:...
### Answer: ...
"""

# source: https://arxiv.org/pdf/2503.19470v1
SYSTEM_PROMPT_1 = """
You are a helpful assistant that can solve the given question step by step. 
Given a question, you need to first think about the reasoning process in
the mind and then provide the answer. The reasoning process and
answer are enclosed within <think> </think> and <answer> </answer> tags respectively. 
For example, <think> This is the reasoning process. </think>
<answer> The final answer is \boxed{answer here}
</answer>. In the last part of the answer, the final exact answer is enclosed within \boxed{}
with xml format.
"""

SYSTEM_PROMPT_2 = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant "
    "first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning "
    "process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., "
    "<think> reasoning process here </think><answer> answer here </answer>"
)

SYSTEM_PROMPT_2_SMALL = """
You are a helpful assistant expert in Unimarc/XML bibliographic records. 
Given a question, you need to first think about the reasoning process in the mind and then provide the answer. 
The reasoning process and answer are separated in distincts paragraphs respectively starting with `### Reasoning:` and `### Answer:`.
For example, ### Reasoning: This is the reasoning process. \n ### Answer: The final exact answer is \boxed{answer here}. 
In the last part of the answer, the final exact answer is enclosed within \boxed{} with XML format.
"""


user_prompt = """
Generate a valid Unimarc/XML record from these unstructured informations:
Title: Electric vehicle tribology
Subtitle: Challenges and opportunities for a sustainable transportation future
Author: Leonardo I. Farfan-Cabrera, Ali Erdemir
Publisher: Elsevier
Year: 2024
ISBN: 978-0-443-14074-7
Language: English
Collection/Series: Elsevier Series on Tribology and Surface Engineering
Edition: Not specified
Material description: 1 vol. (XI-313 p.), couv. ill. en coul., 23 cm
Abstract/Notes: "Electric vehicle tribology, challenges and opportunities for a sustainable transportation future" provides practical, comprehensive guidance on a new and increasingly important area of tribology. Building skills from fundamentals to solution design, this book demonstrates the unique tribological techniques essential to the efficient electrification of transport systems. Led by professors with a combined three decades in industry and academia, and collecting insights from experts around the world, this book begins with the essential knowledge regarding both electric vehicles and tribology. After outlining the unique tribological needs of EVs, the book then breaks down the components and hardware required. It provides detailed protocols and methods for the testing and improvement of lubricants and materials as well as a dedicated section on modern lubrication specific to EVs. Throughout, it considers the critical question of sustainable tribology and the long-term sustainable options for lubrication and materials for electric vehicles.
Source of the abstract/notes: 4e de couverture
Table of contents: Not specified
Keywords: Tribologie (technologie), Tribologie (Technologie)
"""

In [11]:
messages = [
  {"role": "system", "content": SYSTEM_PROMPT_2_SMALL},
  {"role": "user", "content": user_prompt},
]

In [12]:
inputs = tokenizer.apply_chat_template(
    messages,
    return_dict=True,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to(device)

In [13]:
outputs = model.generate(
    **inputs,
    max_new_tokens = 2048,
    use_cache = True,
)

In [ ]:
#print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [14]:
generated_ids = [
   output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, outputs)
]

In [15]:
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

### Reasoning:

To generate a valid Unimarc/XML record, we need to follow these steps:

1. **Identify the Document Type**: The document type should be `Book`. Since the title, author, publisher, year, ISBN, language, and collection series are provided, we can infer that the document is likely a book.

2. **Extract Title**: From the title "Electric vehicle tribology", we can extract the following information:
   - Book Title: Electric vehicle tribology
   - Author: Leonardo I. Farfan-Cabrera, Ali Erdemir
   - Publisher: Elsevier
   - Year: 2024
   - ISBN: 978-0-443-14074-7
   - Language: English

3. **Extract Publication Information**: From the publication information "Elsevier", we can extract the following details:
   - Source: 4e de couverture
   - Collection: Elsevier Series on Tribology and Surface Engineering
   - Edition: Not specified

4. **Extract Abstract/Notes**: The abstract/notes section does not contain any explicit text but suggests that there might be additional informat

# Load data

In [24]:
from datasets import load_dataset, Dataset
import pandas as pd

In [25]:
dataset = load_dataset("Geraldine/Unimarc-iln050-5k", split="train")
dataset

Generating train split: 100%|██████████| 5129/5129 [00:00<00:00, 10334.49 examples/s]


Dataset({
    features: ['ppn', 'question', 'answer'],
    num_rows: 5129
})

In [27]:
system_message = """You are a helpful assistant expert in Unimarc/XML bibliographic records. Given bibliographic metadata, generate a valid Unimarc XML record."""
def create_conversation(row):
  return {
    "messages": [
      {"role": "system", "content": system_message},
      {"role": "user", "content": row["question"]},
      {"role": "assistant", "content": row["answer"] + tokenizer.eos_token} # to avoid infinite generation
    ]
  }

dataset = dataset.map(create_conversation, remove_columns=dataset.features, batched=False)
dataset

Map: 100%|██████████| 5129/5129 [00:01<00:00, 3031.70 examples/s]


Dataset({
    features: ['messages'],
    num_rows: 5129
})

In [28]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant expert in Unimarc/XML bibliographic records. Given bibliographic metadata, generate a valid Unimarc XML record.',
   'role': 'system'},
  {'content': 'Title: Les Jeux olympiques de 1892 à 2024\nSubtitle: une aventure mondiale\nAuthor: Patrick Clastres\nAdditional author info: Clastres, Patrick (19..-....)\nPublisher: Presses universitaires de Rennes\nPlace of publication: Rennes\nYear: 2025 (DL 2025)\nISBN: 978-2-7535-9645-0 (br.)\nLanguage: French (fre)\nEdition: Not explicitly stated\nMaterial description: 1 volume (463 pages); color illustrated cover; 24 cm\nCollection/Series: Collection "Histoire"\nSeries ISSN: 1255-2364 (Histoire (Rennes))\nTable of contents: Not provided\nAbstract/Notes: \n"Depuis leur réinvention à Paris et à Athènes à la fin du XIXe siècle, les Jeux olympiques semblent refléter la marche du monde. En fait, ils progressent à leur propre rythme, quadriennal, et ils ne sont jamais qu’un interlude posé dans les

# Lora SFT fine-tuning

In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig

In [29]:
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp
model.layers.2.mlp.gate_proj
model.l

In [ ]:
help(SFTTrainer)

In [31]:
# ---- 3. Define LoRA config ----
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"] # may vary based on model, check with `model.named_modules()` in previous cell
)

# ---- 4. Training args ----
training_args = TrainingArguments(
    output_dir="./sft_lora_output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    save_strategy="epoch",
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

# ---- 5. Launch training ----
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args
)

/usr/local/lib/python3.12/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
Truncating train dataset: 100%|██████████| 5129/5129 [00:02<00:00, 1934.43 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [32]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.32 GiB. GPU 0 has a total capacity of 14.57 GiB of which 292.75 MiB is free. Process 3526258 has 14.28 GiB memory in use. Of the allocated memory 11.58 GiB is allocated by PyTorch, and 2.56 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [22]:
trainer.save_model()

In [23]:
peft_model_id = "sft_lora_output"
tr_model_id = "Qwen/Qwen2.5-0.5B-Instruct"

In [24]:
from peft import AutoPeftModelForCausalLM, PeftModel

model = AutoModelForCausalLM.from_pretrained(tr_model_id, trust_remote_code=True, torch_dtype=torch.float16,
     low_cpu_mem_usage=True,)
model = PeftModel.from_pretrained(model, peft_model_id)
merged_model = model.merge_and_unload()

In [25]:
merged_model.push_to_hub(f"Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc")

model.safetensors: 100%|██████████| 988M/988M [00:52<00:00, 18.8MB/s] 


CommitInfo(commit_url='https://huggingface.co/Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc/commit/4bb9dfb8545d1dc0db42e35a8222c3f92022d08f', commit_message='Upload Qwen2ForCausalLM', commit_description='', oid='4bb9dfb8545d1dc0db42e35a8222c3f92022d08f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc', endpoint='https://huggingface.co', repo_type='model', repo_id='Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc'), pr_revision=None, pr_num=None)

In [26]:
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer.push_to_hub(f"Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc")

tokenizer.json: 100%|██████████| 11.4M/11.4M [00:00<00:00, 25.0MB/s]


CommitInfo(commit_url='https://huggingface.co/Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc/commit/9165c226214411fc1714214a4681f425d8ee742e', commit_message='Upload tokenizer', commit_description='', oid='9165c226214411fc1714214a4681f425d8ee742e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc', endpoint='https://huggingface.co', repo_type='model', repo_id='Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc'), pr_revision=None, pr_num=None)

## Inference with Lora fine-tuned model

In [27]:
model_name = "Geraldine/FineQwen2.5-0.5B-Instruct-sft-unimarc"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    #attn_implementation="flash_attention_2"
)
model.config.sliding_window = None

In [28]:
model.config.sliding_window = None
device = "cuda" # for GPU usage or "cpu" for CPU usage
model = model.to(device)
model.eval()

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [39]:
messages = [
  {"role": "system", "content": SYSTEM_PROMPT_0_SMALL},
  {"role": "user", "content": user_prompt},
]

In [40]:
inputs = tokenizer.apply_chat_template(
    messages,
    return_dict=True,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to(device)

In [41]:
outputs = model.generate(
    **inputs,
    max_new_tokens = 4096,
    repetition_penalty=1.2,
    use_cache = True,
)

In [43]:
generated_ids = [
   output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, outputs)
]

In [44]:
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

<record><controlfield tag="005">20250314162408.000</controlfield><leader> cam0 22        450 </leader><controlfield tag="001">27698450X</controlfield><controlfield tag="003">http://www.sudoc.fr/27698450X</controlfield><datafield tag="035" ind1=" " ind2=" "><subfield code="a">(OCoLC)1423210952</subfield></datafield><datafield tag="010" ind1=" " ind2=" "><subfield code="a">978-0-443-14074-7</subfield><subfield code="b">br.</subfield></datafield><datafield tag="073" ind1=" " ind2="1"><subfield code="a">9780443140747</subfield></datafield><datafield tag="100" ind1=" " ind2=" "><subfield code="a">20240411h20242024k  y0frey50      ba</subfield></datafield><datafield tag="101" ind1="0" ind2=" "><subfield code="a">eng</subfield><subfield code="d">eng</subfield><subfield code="f">eng</subfield><subfield code="g">eng</subfield><subfield code="h">eng</subfield><subfield code="i">eng</subfield><subfield code="n">eng</subfield><subfield code="z">eng</subfield></datafield><datafield tag="105" ind1="

# Reward functions

## utils functions

In [29]:
import xml.etree.ElementTree as ET
import re

def extract_hash_answer(text: str) -> str | None:
    if "### Answer:" not in text:
        return None
    first_split = text.split("### Answer:")[1].strip()
    if "### Explanation:" not in first_split:
        return first_split
    else:
        return first_split.split("### Explanation:")[0].strip()

def extract_xml(text: str) -> str | None:
    # Use regular expression to find the XML part enclosed in ```xml...```
    xml_match = re.search(r'```xml(.*?)```', response, re.DOTALL)
    if xml_match:
        return xml_match.group(1).strip()
    else:
        return None

def extract_field_values(xml_str):
    root = ET.fromstring(xml_str)
    fields = {}
    for df in root.findall(".//datafield"):
        tag = df.get("tag")
        subfields = [sf.text for sf in df.findall("subfield")]
        fields[tag] = " ".join(subfields)
    return fields

In [24]:
# Example avec previous example
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [25]:
print(extract_hash_answer(text=response))

```xml
<?xml version="1.0"?>
<record xmlns="http://unimarc.org/xml/ns/unimarc">
    <author>
        <name>Leonardo I. Farfan-Cabrera, Ali Erdemir</name>
    </author>
    <publisher>Elsevier</publisher>
    <year>2024</year>
    <isbn>978-0-443-14074-7</isbn>
    <language>English</language>
    <collection>
        <series>Elsevier Series on Tribology and Surface Engineering</series>
    </collection>
    <edition>
        Not specified
    </edition>
    <material>
        <description>1 vol. (XII-313 p.), couv. ill. en coul., 23 cm</description>
    </material>
    <abstract>
        "Electric vehicle tribology, challenges and opportunities for a sustainable transportation future"
        provides practical, comprehensive guidance on a new and increasingly important area of tribology. Building skills from fundamentals to solution design, this book demonstrates the unique tribological techniques essential to the efficient electrification of transport systems."
    </abstract>
    <k

In [26]:
print(extract_xml(text=response))

<?xml version="1.0"?>
<record xmlns="http://unimarc.org/xml/ns/unimarc">
    <author>
        <name>Leonardo I. Farfan-Cabrera, Ali Erdemir</name>
    </author>
    <publisher>Elsevier</publisher>
    <year>2024</year>
    <isbn>978-0-443-14074-7</isbn>
    <language>English</language>
    <collection>
        <series>Elsevier Series on Tribology and Surface Engineering</series>
    </collection>
    <edition>
        Not specified
    </edition>
    <material>
        <description>1 vol. (XII-313 p.), couv. ill. en coul., 23 cm</description>
    </material>
    <abstract>
        "Electric vehicle tribology, challenges and opportunities for a sustainable transportation future"
        provides practical, comprehensive guidance on a new and increasingly important area of tribology. Building skills from fundamentals to solution design, this book demonstrates the unique tribological techniques essential to the efficient electrification of transport systems."
    </abstract>
    <keywords

## Format-based reward

In [27]:
def format_reward(xml_output: str) -> float:
    try:
        root = ET.fromstring(xml_output)
    except ET.ParseError:
        return 0.0  # Not even valid XML

    required_tags = ["leader", "controlfield", "datafield"]
    has_required_tags = all(root.find(f".//{tag}") is not None for tag in required_tags)

    if not has_required_tags:
        return 0.3  # Barely structured but not fully compliant

    # Optionally: integrate XSD/Schematron validation here
    return 1.0  # Well-formed and conforms to expected structure

In [28]:
xml_answer = extract_xml(text=response)
print(format_reward(xml_output=xml_answer))

0.3


## Accuracy-based reward

In [30]:
xml_answer = extract_xml(text=response)
xml_target = """<record><controlfield tag="005">20240711151255.000</controlfield><leader> nam0 22 450 </leader><controlfield tag="001">279354177</controlfield><controlfield tag="003">http://www.sudoc.fr/279354177</controlfield><datafield tag="035" ind1=" " ind2=" "><subfield code="a">(OCoLC)1449675800</subfield></datafield><datafield tag="010" ind1=" " ind2=" "><subfield code="a">978-0-443-14074-7</subfield><subfield code="b">br.</subfield></datafield><datafield tag="073" ind1=" " ind2="1"><subfield code="a">9780443140747</subfield></datafield><datafield tag="100" ind1=" " ind2=" "><subfield code="a">20240711d2024 ||||0frey50 ba</subfield></datafield><datafield tag="101" ind1="0" ind2=" "><subfield code="a">eng</subfield><subfield code="2">639-2</subfield></datafield><datafield tag="105" ind1=" " ind2=" "><subfield code="a">y ||||001yy</subfield></datafield><datafield tag="106" ind1=" " ind2=" "><subfield code="a">r</subfield></datafield><datafield tag="181" ind1=" " ind2=" "><subfield code="6">z01</subfield><subfield code="c">txt</subfield><subfield code="2">rdacontent</subfield></datafield><datafield tag="181" ind1=" " ind2="1"><subfield code="6">z01</subfield><subfield code="a">i#</subfield><subfield code="b">xxxe##</subfield></datafield><datafield tag="182" ind1=" " ind2=" "><subfield code="6">z01</subfield><subfield code="c">c</subfield><subfield code="2">rdamedia</subfield></datafield><datafield tag="182" ind1=" " ind2="1"><subfield code="6">z01</subfield><subfield code="a">b</subfield></datafield><datafield tag="183" ind1=" " ind2="1"><subfield code="6">z01</subfield><subfield code="a">nga</subfield><subfield code="2">RDAfrCarrier</subfield></datafield><datafield tag="102" ind1=" " ind2=" "><subfield code="a">NL</subfield><subfield code="a">GB</subfield><subfield code="a">US</subfield></datafield><datafield tag="200" ind1="0" ind2=" "><subfield code="a">Electric vehicle tribology</subfield><subfield code="e">challenges and opportunities for a sustainable transportation future</subfield><subfield code="f">edited by Leonardo I. Farfan-Cabrera, ... Ali Erdemir, ...</subfield></datafield><datafield tag="701" ind1=" " ind2="1"><subfield code="3">279354606</subfield><subfield code="a">Farfan-Cabrera</subfield><subfield code="b">Leonardo Israel</subfield><subfield code="4">340</subfield></datafield><datafield tag="701" ind1=" " ind2="1"><subfield code="3">11490314X</subfield><subfield code="a">Erdemir</subfield><subfield code="b">Ali</subfield><subfield code="4">340</subfield></datafield><datafield tag="801" ind1=" " ind2="3"><subfield code="a">FR</subfield><subfield code="b">Abes</subfield><subfield code="c">20240711</subfield><subfield code="g">AFNOR</subfield></datafield><datafield tag="214" ind1=" " ind2="0"><subfield code="a">Amsterdam</subfield><subfield code="c">Elsevier</subfield><subfield code="d">2024</subfield></datafield><datafield tag="215" ind1=" " ind2=" "><subfield code="a">1 vol. (XI-313 p.)</subfield><subfield code="c">couv. ill. en coul.</subfield><subfield code="d">23 cm</subfield></datafield><datafield tag="225" ind1="0" ind2=" "><subfield code="a">Elsevier Series on Tribology and Surface Engineering</subfield></datafield><datafield tag="606" ind1=" " ind2=" "><subfield code="3">027720055</subfield><subfield code="a">Tribologie (technologie)</subfield><subfield code="2">rameau</subfield></datafield><datafield tag="606" ind1=" " ind2=" "><subfield code="a">Tribologie (Technologie)</subfield><subfield code="2">lc</subfield></datafield><datafield tag="330" ind1=" " ind2=" "><subfield code="a">"Electric vehicle tribology, challenges and opportunities for a sustainable transportation future" provides practical, comprehensive guidance on a new and increasingly important aera of tribology. Building skills from fundamentals to solution design, this book demonstrates the unique tribological techniques essential to the efficient electrification of transportt systems. Led by professors with a combined three decades in industry and academia, and collecting insights from experts around the world, this book begins with the essential knowledge regarding both electric vehicles and tribology. After outlining the unique tribological needs of EVs, the book then breaks down the components and hardware required. It provides detailed protocols and methods for the testing and improvement of lubricants and materials as well as a dedicated section on modern lubrication specific to EVs. Throughout, it considers the critical question of sustainable tribology and the long-term sustainable options for lubrication and materials for electric vehicles.</subfield><subfield code="2">4e de couverture</subfield></datafield></record>"""

In [34]:
print(extract_field_values(xml_answer))

{}


In [33]:
print(extract_field_values(xml_target))

{'035': '(OCoLC)1449675800', '010': '978-0-443-14074-7 br.', '073': '9780443140747', '100': '20240711d2024 ||||0frey50 ba', '101': 'eng 639-2', '105': 'y ||||001yy', '106': 'r', '181': 'z01 i# xxxe##', '182': 'z01 b', '183': 'z01 nga RDAfrCarrier', '102': 'NL GB US', '200': 'Electric vehicle tribology challenges and opportunities for a sustainable transportation future edited by Leonardo I. Farfan-Cabrera, ... Ali Erdemir, ...', '701': '11490314X Erdemir Ali 340', '801': 'FR Abes 20240711 AFNOR', '214': 'Amsterdam Elsevier 2024', '215': '1 vol. (XI-313 p.) couv. ill. en coul. 23 cm', '225': 'Elsevier Series on Tribology and Surface Engineering', '606': 'Tribologie (Technologie) lc', '330': '"Electric vehicle tribology, challenges and opportunities for a sustainable transportation future" provides practical, comprehensive guidance on a new and increasingly important aera of tribology. Building skills from fundamentals to solution design, this book demonstrates the unique tribological te

In [31]:
def accuracy_reward(generated_xml: str, target_xml: str) -> float:
    try:
        gen_fields = extract_field_values(generated_xml)
        tgt_fields = extract_field_values(target_xml)
    except ET.ParseError:
        return 0.0

    shared_keys = set(gen_fields) & set(tgt_fields)
    if not shared_keys:
        return 0.0

    total_sim = 0
    for key in shared_keys:
        sim = SequenceMatcher(None, gen_fields[key], tgt_fields[key]).ratio()
        total_sim += sim

    return total_sim / len(shared_keys)

In [32]:
print(accuracy_reward(generated_xml=xml_answer, target_xml=xml_target))

0.0
